In [6]:
# from reso_extract import ms_graph
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
from pathlib import Path

# Define the path you want to add
new_path = Path("C:\\Users\\andmn\\PycharmProjects\\kverse")

# Convert Path object to string and append it to sys.path
sys.path.append(str(new_path))
from selenium.webdriver.support.wait import WebDriverWait
from kdags.assets.reparation.reso_old.web_driver import initialize_driver, login_to_reso
from kdags.assets.reparation.reso_old.document_extractor import (
    extract_docs_data,
    click_see_service_order,
)
from kdags.assets.reparation.reso_old.file_processor import process_docs_data
from kdags.assets.reparation.reso_old.web_utils import (
    wait_for_page_load,
    ensure_clickable,
)
from kdags.assets.reparation.reso_old.file_utils import save_file_locally
from kdags.assets.reparation.reso_old.config import DEFAULT_WAIT
import pandas as pd
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
import dagster as dg

context = dg.build_asset_context()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import time
from datetime import datetime
from pathlib import Path

# --- Dagster Imports ---
import dagster as dg
import pandas as pd
import polars as pl

# --- Selenium Imports ---
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from kdags.assets.reparation.reso_old.config import DEFAULT_WAIT
from kdags.assets.reparation.reso_old.web_driver import initialize_driver, login_to_reso
from kdags.resources.tidyr import (
    DataLake,
)  # Assuming DataLake is initialized via resources or globally accessible


def click_reportabilidad(driver, wait):
    """Clicks the main 'Reportabilidad' section."""
    print("--- Clicking Reportabilidad ---")
    reportabilidad_locator = (
        By.XPATH,
        "//span[normalize-space(text())='Reportabilidad']",
    )
    reportabilidad_span = wait.until(
        EC.presence_of_element_located(reportabilidad_locator)
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", reportabilidad_span)
    wait.until(EC.element_to_be_clickable(reportabilidad_locator))
    reportabilidad_span.click()
    print("Clicked 'Reportabilidad'.")


def click_component_status(driver, wait):
    """Clicks the 'Estatus Componente' sub-link."""
    estatus_componente_locator = (By.ID, "submodulo-369")
    estatus_componente_link = wait.until(
        EC.presence_of_element_located(estatus_componente_locator)
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", estatus_componente_link)
    wait.until(EC.element_to_be_clickable(estatus_componente_locator))
    estatus_componente_link.click()

    time.sleep(1)


def set_default_component_status_filters(driver, wait: WebDriverWait):
    """Sets the default filters: Toggles 'See All', selects all workshops."""
    # print("Setting default component status filters...")

    # --- NEW: Wait for filters to finish loading ---
    # print("Waiting for filter elements to be populated...")
    # Wait for the 'Select All Workshops' checkbox to be present in the DOM.
    # Its presence indicates the Workshop multi-select has been loaded by JS.
    select_all_locator = (By.ID, "cbxTodosTalleres")
    wait.until(EC.presence_of_element_located(select_all_locator))
    # Optionally, wait for another indicator like a label text
    # wait.until(EC.text_to_be_present_in_element((By.ID, 'lblTaller'), 'Workshop'))
    # print("Filter elements appear to be loaded.")
    # Add a very small static pause *just in case* there's a tiny delay
    # between presence and full readiness for interaction, although EC should handle this.
    time.sleep(0.5)

    # 1. Click "See All" switch
    # print("Locating 'See All' switch...")
    see_all_switch_locator = (By.CSS_SELECTOR, "input#verTodosOS + span.switchery")
    # Use presence first, then check clickability after scroll
    see_all_switch = wait.until(EC.presence_of_element_located(see_all_switch_locator))
    # print("'See All' switch found. Scrolling into view...")
    driver.execute_script(
        "arguments[0].scrollIntoView({block: 'center'});", see_all_switch
    )
    # Now wait for clickability
    clickable_switch = wait.until(EC.element_to_be_clickable(see_all_switch_locator))
    # print("Clicking 'See All' switch...")
    clickable_switch.click()
    # print("'See All' switch clicked.")

    # 2. Open Workshop dropdown (using original overlay locator)
    # print("Locating Workshop dropdown overlay...")
    # --- Using the original overlay locator as requested ---
    overlay_locator = (
        By.XPATH,
        "//select[@id='cbxModeloDTaller']/following-sibling::div[contains(@class, 'overSelect')]",
    )
    select_element_locator = (By.ID, "cbxModeloDTaller")  # For scrolling reference

    # Ensure the underlying select element exists before scrolling
    select_element = wait.until(EC.presence_of_element_located(select_element_locator))

    # print("Scrolling Workshop dropdown area into view...")
    driver.execute_script(
        "arguments[0].scrollIntoView({block: 'center'});", select_element
    )

    # Wait for the overlay element itself to be present before trying to click
    overlay_div = wait.until(EC.presence_of_element_located(overlay_locator))

    # --- IMPORTANT: Wait for the overlay to be clickable ---
    # Even if present, it might not be immediately clickable
    wait.until(EC.element_to_be_clickable(overlay_locator))

    # print("Clicking Workshop dropdown overlay...")
    overlay_div.click()  # Click the overlay div
    # print("Workshop dropdown overlay clicked.")

    # 3. Click "SELECCIONAR TODOS" checkbox inside the dropdown
    # print("Locating 'Select All Workshops' checkbox...")
    # select_all_locator is already defined from the initial wait
    # Wait for the checkbox to become VISIBLE now that the dropdown is open
    wait.until(EC.visibility_of_element_located(select_all_locator))
    # print("'Select All Workshops' checkbox is visible. Waiting for clickability...")
    select_all_checkbox = wait.until(EC.element_to_be_clickable(select_all_locator))
    # print("Clicking 'Select All Workshops' checkbox...")
    # Using standard click first
    select_all_checkbox.click()
    # If standard click fails, try JS click:
    # driver.execute_script("arguments[0].click();", select_all_checkbox)
    # print("'Select All Workshops' checkbox clicked.")
    time.sleep(0.5)  # Small pause after click might help stability

    # 4. Close Workshop dropdown (using original overlay locator)
    # print("Attempting to close Workshop dropdown by clicking overlay again...")
    # Re-find and wait for the overlay to be clickable again
    overlay_div_to_close = wait.until(EC.element_to_be_clickable(overlay_locator))
    overlay_div_to_close.click()
    # print("Clicked Workshop overlay again to close.")

    # Wait for the dropdown content (the 'Select All' checkbox) to become invisible
    # print("Waiting for Workshop dropdown to close...")
    wait.until(EC.invisibility_of_element_located(select_all_locator))
    # print("Workshop dropdown closed.")

    # print("Default component status filters set successfully.")


# --- Helper function to handle date setting logic ---
def set_date(
    date_str: str,
    input_locator: tuple,
    icon_locator: tuple,
    date_label: str,
    datepicker_widget_locator,
):
    print(f"Setting '{date_label}': {date_str}")

    # Parse day - Script will raise ValueError here if format is wrong
    target_day_dt = datetime.strptime(date_str, "%d-%m-%Y")
    target_day_number = str(target_day_dt.day)
    print(f"  Target Day: {target_day_number}")

    # --- Interact with Input Field ---
    date_input: WebElement = wait.until(EC.element_to_be_clickable(input_locator))
    driver.execute_script("arguments[0].scrollIntoViewIfNeeded(true);", date_input)
    date_input.clear()
    date_input.send_keys(date_str)
    print(f"  Sent keys '{date_str}' to input field.")
    wait.until(EC.text_to_be_present_in_element_value(input_locator, date_str))
    print("  Input value confirmed.")

    # --- Open and Interact with Calendar Widget ---
    calendar_icon: WebElement = wait.until(EC.element_to_be_clickable(icon_locator))
    # -- Start Change --
    # Replace the standard click with a JavaScript click
    print("  Clicking calendar icon using JavaScript...")
    driver.execute_script("arguments[0].click();", calendar_icon)
    # print("  Clicked calendar icon.") # Keep one print statement
    # -- End Change --

    # Wait explicitly for the calendar widget container to be visible (no change here yet)
    datepicker_widget: WebElement = wait.until(
        EC.visibility_of_element_located(datepicker_widget_locator)
    )
    print("  Calendar widget is visible.")

    # --- Locate and Click the Target Day ---
    # Define the relative locator for searching WITHIN the datepicker widget
    day_link_locator_relative = (
        By.XPATH,
        f".//td[not(contains(@class, 'ui-datepicker-other-month'))]/a[normalize-space(text())='{target_day_number}']",
    )
    print(f"  Looking for day '{target_day_number}' link within the calendar widget...")

    # Step 1: Find the day link element *within the datepicker widget*.
    # We use a wait with a lambda here just to ensure it's present before checking clickability.
    # This lambda correctly calls find_element and returns the WebElement.
    day_link_element: WebElement = wait.until(
        lambda d: datepicker_widget.find_element(*day_link_locator_relative)
    )
    print(
        f"  Found day '{target_day_number}' element. Now waiting for it to be clickable..."
    )

    # Step 2: Wait for the *found WebElement* to be clickable.
    # Pass the actual WebElement variable here, NOT the locator or lambda.
    wait.until(EC.element_to_be_clickable(day_link_element))
    print(f"  Day '{target_day_number}' is clickable.")

    # Step 3: Click the now confirmed clickable element.
    day_link_element.click()
    print(f"  Clicked day '{target_day_number}' in calendar for '{date_label}'.")

    # --- Wait for Calendar to Close ---
    wait.until(EC.invisibility_of_element_located(datepicker_widget_locator))
    print("  Calendar widget closed.")


def set_component_status_dates(
    driver, wait: WebDriverWait, start_date: str, end_date: str
):
    print(f"--- Setting Dates: {start_date} to {end_date} ---")

    # --- Locators ---
    date_from_input_locator = (By.ID, "dfechaInicial")
    calendar_icon_from_locator = (By.CSS_SELECTOR, "span.controlFechaInicial")
    date_up_input_locator = (By.ID, "dfechaFinal")
    calendar_icon_up_locator = (By.CSS_SELECTOR, "span.controlFechaFinal")
    datepicker_widget_locator = (By.ID, "ui-datepicker-div")  # The main calendar pop-up

    # --- Call helper function for both dates ---
    set_date(
        start_date,
        date_from_input_locator,
        calendar_icon_from_locator,
        "Date From",
        datepicker_widget_locator,
    )
    time.sleep(1)
    set_date(
        end_date,
        date_up_input_locator,
        calendar_icon_up_locator,
        "Date Up",
        datepicker_widget_locator,
    )

    print(f"\nFinished setting dates: {start_date} to {end_date}")


def export_component_status_results(driver, wait):
    """Clicks Search and then Export, waiting for results."""
    search_button_locator = (By.CSS_SELECTOR, "span.btn.btnVerde.btnBuscar")
    search_button = wait.until(EC.presence_of_element_located(search_button_locator))
    wait.until(EC.element_to_be_clickable(search_button_locator))
    search_button.click()

    # Wait for loading indicator to disappear
    loading_indicator_locator = (By.CSS_SELECTOR, "div.updateProgress")
    # Use a longer timeout if search is slow
    results_wait = WebDriverWait(driver, 120)  # 60 second timeout for results
    results_wait.until(EC.invisibility_of_element_located(loading_indicator_locator))

    # Click Export to Excel
    export_button_locator = (By.ID, "ContentPlaceHolder1_btnExportar")
    export_button = wait.until(EC.presence_of_element_located(export_button_locator))
    wait.until(EC.element_to_be_clickable(export_button_locator))
    export_button.click()
    time.sleep(5)  # Pause to allow download to initiate/complete


def upload_component_status_results(
    context: dg.AssetExecutionContext, abfs_path: str, year: int
):
    """Finds latest download, reads as HTML table, uploads to ADLS, deletes local file."""
    print(f"--- Uploading results for year {year} to {abfs_path} ---")

    download_dir = Path.home() / "Downloads"
    context.log.info(f"Checking for downloads in: {download_dir}")

    if not download_dir.is_dir():
        context.log.error(f"Download directory not found: {download_dir}")
        return None  # Indicate failure

    time.sleep(1)  # Pause for file system

    excel_files = list(download_dir.glob("*.xlsx")) + list(download_dir.glob("*.xls"))

    excel_files.sort(key=lambda f: f.stat().st_mtime, reverse=True)
    downloaded_file_path = excel_files[0]
    context.log.info(f"Identified most recent Excel file: {downloaded_file_path}")

    df_pandas = pd.read_html(downloaded_file_path)[0]

    context.log.info(f"Successfully read data from {downloaded_file_path.name}")

    df_polars = pl.from_pandas(df_pandas)

    # Upload to Data Lake
    # Assuming DataLake resource 'datalake' is configured for the asset
    # Or initialize here if not using resources: datalake = DataLake()
    datalake = DataLake()  # Initialize directly for this example
    datalake.upload_tibble(abfs_path=abfs_path, df=df_polars, format="parquet")
    context.log.info(f"Successfully uploaded data for year {year} to {abfs_path}")

    # Delete local file
    context.log.info(f"Deleting local file: {downloaded_file_path}")
    downloaded_file_path.unlink(
        missing_ok=True
    )  # missing_ok=True ignores error if already gone
    context.log.info(f"Deleted local file: {downloaded_file_path.name}")

    return abfs_path  # Return the URI of the uploaded file

In [51]:
uploaded_uris = []  # Keep track of successful uploads
summary_data = {
    "years_processed": [],
    "uploaded_files": [],
    "failed_years": [],
    "num_files_uploaded": 0,
    "status": "incomplete",
}

# # --- Initialize Driver ---
context.log.info("Initializing WebDriver...")
driver = initialize_driver()
wait = WebDriverWait(driver, DEFAULT_WAIT)
context.log.info("WebDriver initialized.")

# # --- Login ---
context.log.info("Logging into RESO+...")
login_to_reso(driver, wait)
context.log.info("Login successful.")

# # --- Navigate ---
click_reportabilidad(driver, wait)
click_component_status(driver, wait)
context.log.info("Navigated to Estatus Componente page.")

# --- Set Default Filters ---
set_default_component_status_filters(driver, wait)
context.log.info("Default filters applied.")

# --- Process Past Years ---
num_years_back = 2  # Define how many years back to process
current_date = datetime.now()
current_year = current_date.year
start_loop_year = current_year
end_loop_year = start_loop_year - num_years_back

context.log.info(
    f"Starting year loop from {start_loop_year} down to {end_loop_year}..."
)

for year in range(start_loop_year, end_loop_year - 1, -1):
    context.log.info(f"Processing Year: {year}")
    summary_data["years_processed"].append(year)
    start_date_str = f"01-01-{year}"
    end_date_str = f"31-12-{year}"

    # Set dates for the current year
    set_component_status_dates(driver, wait, start_date_str, end_date_str)

    # Export the results for the current year
    # export_component_status_results(driver, wait)

    # Define upload URI for the current year
    # Using current_date for partition, but year for filename seems intended
    partition_path = current_date.strftime(
        "y=%Y/m=%m/d=%d"
    )  # Partition based on run date
    abfs_path = f"abfs://bhp-raw-data/RESO/COMPONENT_STATUS/{partition_path}/component_status_{year}.parquet"

    # # Upload results and cleanup
    # uploaded_uri = upload_component_status_results(context, abfs_path, year)
    # if uploaded_uri:
    #     uploaded_uris.append(uploaded_uri)
    #     # Optionally add more details per file if needed
    #     summary_data["uploaded_files"].append({"year": year, "abfs_path": uploaded_uri})
    # else:
    #     context.log.error(f"Upload failed for year {year}")
    #     summary_data["failed_years"].append(year)
    #     # Decide if you want to stop or continue if one year fails

context.log.info("Year loop finished.")
summary_data["status"] = (
    "completed" if not summary_data["failed_years"] else "completed_with_errors"
)
summary_data["num_files_uploaded"] = len(uploaded_uris)

context.log.info("Closing WebDriver...")
driver.quit()
context.log.info("WebDriver closed.")

# Return the summary dictionary

2025-04-06 19:16:55 -0400 - dagster - INFO - system - Initializing WebDriver...
2025-04-06 19:16:56 -0400 - dagster - INFO - system - WebDriver initialized.
2025-04-06 19:16:56 -0400 - dagster - INFO - system - Logging into RESO+...
2025-04-06 19:17:18 -0400 - dagster - INFO - system - Login successful.


--- Clicking Reportabilidad ---
Clicked 'Reportabilidad'.


2025-04-06 19:17:30 -0400 - dagster - INFO - system - Navigated to Estatus Componente page.
2025-04-06 19:17:31 -0400 - dagster - INFO - system - Default filters applied.
2025-04-06 19:17:31 -0400 - dagster - INFO - system - Starting year loop from 2025 down to 2023...
2025-04-06 19:17:31 -0400 - dagster - INFO - system - Processing Year: 2025


--- Setting Dates: 01-01-2025 to 31-12-2025 ---
Setting 'Date From': 01-01-2025
  Target Day: 1
  Sent keys '01-01-2025' to input field.
  Input value confirmed.
  Clicking calendar icon using JavaScript...
  Calendar widget is visible.
  Looking for day '1' link within the calendar widget...
  Found day '1' element. Now waiting for it to be clickable...
  Day '1' is clickable.
  Clicked day '1' in calendar for 'Date From'.
  Calendar widget closed.
Setting 'Date Up': 31-12-2025
  Target Day: 31
  Sent keys '31-12-2025' to input field.
  Input value confirmed.
  Clicking calendar icon using JavaScript...
  Calendar widget is visible.
  Looking for day '31' link within the calendar widget...
  Found day '31' element. Now waiting for it to be clickable...
  Day '31' is clickable.
  Clicked day '31' in calendar for 'Date Up'.
  Calendar widget closed.

Finished setting dates: 01-01-2025 to 31-12-2025


2025-04-06 19:17:33 -0400 - dagster - INFO - system - Processing Year: 2024


--- Setting Dates: 01-01-2024 to 31-12-2024 ---
Setting 'Date From': 01-01-2024
  Target Day: 1
  Sent keys '01-01-2024' to input field.
  Input value confirmed.
  Clicking calendar icon using JavaScript...
  Calendar widget is visible.
  Looking for day '1' link within the calendar widget...
  Found day '1' element. Now waiting for it to be clickable...
  Day '1' is clickable.
  Clicked day '1' in calendar for 'Date From'.
  Calendar widget closed.
Setting 'Date Up': 31-12-2024
  Target Day: 31
  Sent keys '31-12-2024' to input field.
  Input value confirmed.
  Clicking calendar icon using JavaScript...
  Calendar widget is visible.
  Looking for day '31' link within the calendar widget...
  Found day '31' element. Now waiting for it to be clickable...
  Day '31' is clickable.
  Clicked day '31' in calendar for 'Date Up'.
  Calendar widget closed.

Finished setting dates: 01-01-2024 to 31-12-2024


2025-04-06 19:17:35 -0400 - dagster - INFO - system - Processing Year: 2023


--- Setting Dates: 01-01-2023 to 31-12-2023 ---
Setting 'Date From': 01-01-2023
  Target Day: 1
  Sent keys '01-01-2023' to input field.
  Input value confirmed.
  Clicking calendar icon using JavaScript...
  Calendar widget is visible.
  Looking for day '1' link within the calendar widget...
  Found day '1' element. Now waiting for it to be clickable...
  Day '1' is clickable.
  Clicked day '1' in calendar for 'Date From'.
  Calendar widget closed.
Setting 'Date Up': 31-12-2023
  Target Day: 31
  Sent keys '31-12-2023' to input field.
  Input value confirmed.
  Clicking calendar icon using JavaScript...
  Calendar widget is visible.
  Looking for day '31' link within the calendar widget...
  Found day '31' element. Now waiting for it to be clickable...
  Day '31' is clickable.
  Clicked day '31' in calendar for 'Date Up'.
  Calendar widget closed.

Finished setting dates: 01-01-2023 to 31-12-2023


2025-04-06 19:17:38 -0400 - dagster - INFO - system - Year loop finished.
2025-04-06 19:17:38 -0400 - dagster - INFO - system - Closing WebDriver...
2025-04-06 19:17:40 -0400 - dagster - INFO - system - WebDriver closed.


In [35]:
# # --- Navigate ---
click_reportabilidad(driver, wait)
click_component_status(driver, wait)
context.log.info("Navigated to Estatus Componente page.")

# --- Set Default Filters ---
set_default_component_status_filters(driver, wait)
context.log.info("Default filters applied.")

--- Clicking Reportabilidad ---
Clicked 'Reportabilidad'.


2025-04-06 18:26:13 -0400 - dagster - INFO - system - Navigated to Estatus Componente page.


Setting default component status filters...
Waiting for filter elements to be populated...
Filter elements appear to be loaded.
Locating 'See All' switch...
'See All' switch found. Scrolling into view...
Clicking 'See All' switch...
'See All' switch clicked.
Locating Workshop dropdown overlay...
Scrolling Workshop dropdown area into view...
Clicking Workshop dropdown overlay...
Workshop dropdown overlay clicked.
Locating 'Select All Workshops' checkbox...
'Select All Workshops' checkbox is visible. Waiting for clickability...
Clicking 'Select All Workshops' checkbox...
'Select All Workshops' checkbox clicked.
Attempting to close Workshop dropdown by clicking overlay again...
Clicked Workshop overlay again to close.
Waiting for Workshop dropdown to close...
Workshop dropdown closed.
Default component status filters set successfully.


2025-04-06 18:26:19 -0400 - dagster - INFO - system - Default filters applied.


In [23]:
set_component_status_dates(driver, wait, start_date_str, end_date_str)

--- Setting Dates: 01-01-2025 to 31-12-2025 ---
Setting 'Date From': 01-01-2025
  Target Day: 1
  Sent keys '01-01-2025' to input field.
  Input value confirmed.
  Clicked calendar icon.
  Calendar widget is visible.
  Looking for day '1' link within the calendar widget...
  Found day '1' element. Now waiting for it to be clickable...
  Day '1' is clickable.
  Clicked day '1' in calendar for 'Date From'.
  Calendar widget closed.
Setting 'Date Up': 31-12-2025
  Target Day: 31
  Sent keys '31-12-2025' to input field.
  Input value confirmed.
  Clicked calendar icon.
  Calendar widget is visible.
  Looking for day '31' link within the calendar widget...
  Found day '31' element. Now waiting for it to be clickable...
  Day '31' is clickable.
  Clicked day '31' in calendar for 'Date Up'.
  Calendar widget closed.

Finished setting dates: 01-01-2025 to 31-12-2025


In [57]:
DataLake().list_paths("abfs://bhp-raw-data/RESO/COMPONENT_STATUS").to_dict()

{'uri': shape: (19,)
 Series: 'uri' [str]
 [
 	"abfs://bhp-raw-data/RESO/COMPO…
 	"abfs://bhp-raw-data/RESO/COMPO…
 	"abfs://bhp-raw-data/RESO/COMPO…
 	"abfs://bhp-raw-data/RESO/COMPO…
 	"abfs://bhp-raw-data/RESO/COMPO…
 	…
 	"abfs://bhp-raw-data/RESO/COMPO…
 	"abfs://bhp-raw-data/RESO/COMPO…
 	"abfs://bhp-raw-data/RESO/COMPO…
 	"abfs://bhp-raw-data/RESO/COMPO…
 	"abfs://bhp-raw-data/RESO/COMPO…
 ],
 'file_size': shape: (19,)
 Series: 'file_size' [i64]
 [
 	47311
 	432119
 	584718
 	534637
 	588865
 	…
 	745991
 	767076
 	225334
 	848581
 	225334
 ],
 'last_modified': shape: (19,)
 Series: 'last_modified' [datetime[μs]]
 [
 	2025-04-03 23:19:31
 	2025-04-03 23:19:35
 	2025-04-03 23:19:42
 	2025-04-03 23:19:46
 	2025-04-03 23:19:50
 	…
 	2025-04-03 23:20:38
 	2025-04-04 01:41:38
 	2025-04-04 01:40:04
 	2025-04-04 01:38:25
 	2025-04-04 01:36:32
 ]}

In [3]:
service_order_df = pd.read_excel(
    r"C:\Users\andmn\PycharmProjects\kbooks\para_descargar.xlsx"
)
service_order_df

,service_order,equipment_name,customer_work_order,component_serial,changeout_date,subcomponent_name
0,14140838,TK399,18123419,EE13070408,2023-10-30,alternador_principal
1,14141425,TK859,18123759,EE13090440,2023-11-08,alternador_principal
2,14140979,TK883,18123800,EE14010717,2023-12-01,alternador_principal
3,14142021,TK852,18123924,EE19090298,2023-12-09,alternador_principal
4,14143179,TK868,18125034,EE14090173,2024-02-07,alternador_principal
5,14143655,TK851,18125407,EE14070623,2024-03-14,alternador_principal
6,14144274,TK849,18125653,EE14030724,2024-03-28,alternador_principal
7,14146810,TK860,18127630,EE11060983A,2024-04-03,alternador_principal
8,14144747,TK285,18126171,EE04090417,2024-04-10,alternador_principal
9,14144858,TK848,18126402,EE08080858,2024-04-24,alternador_principal


In [8]:
# Ensure the 'processed' column exists (default False)
if "processed" not in service_order_df.columns:
    service_order_df["processed"] = False


# Iterate over service orders that are not yet processed
for idx, ir in service_order_df.iterrows():
    if ir.processed:
        continue  # Skip already processed rows

    print(f"\nProcessing Row {idx} for Service Order: {ir.service_order}")

    # Wait for page to be fully loaded
    wait_for_page_load(wait)

    # Wait for and enter the service order code
    input_field = wait.until(EC.presence_of_element_located((By.ID, "codigoBuscar")))
    input_field.clear()
    input_field.send_keys(ir.service_order)

    # Click search button
    search_button = wait.until(EC.element_to_be_clickable((By.ID, "buttonBuscar")))
    ensure_clickable(driver, search_button)
    search_button.click()

    # Wait for results to load
    wait_for_page_load(wait)
    wait.until(EC.presence_of_element_located((By.ID, "WebGrid")))

    # Click the 'See service order' button
    click_see_service_order(wait)

    # Wait for page load after clicking service order
    wait_for_page_load(wait)

    # Click the Documents tab and wait for it to load
    docs_tab = wait.until(EC.element_to_be_clickable((By.ID, "v-pills-documentos-tab")))
    ensure_clickable(driver, docs_tab)
    docs_tab.click()
    wait_for_page_load(wait)

    # Extract the docs data from the documents section
    docs_data = extract_docs_data(driver, wait)

    # Skip processing if no documents were found
    if docs_data is None:
        print(
            f"No documents found for service order {ir.service_order} - marking as processed"
        )
        service_order_df.loc[idx, "processed"] = True
        continue

    # Process the extracted docs data
    processed_df = process_docs_data(
        docs_data, ir.subcomponent_name, ir.component_serial
    ).query("file_type=='INFORMES'")

    # Iterate over each document row and upload
    for jdx, row in processed_df.iterrows():
        result_url = save_file_locally(
            target_folder=row["target_folder"],
            file_name=row["file_name"],
            blob_url=row["blob_url"],
        )
        print(f"Document {jdx}: Uploaded successfully: {result_url}")

    # Click the close button
    close_button = wait.until(EC.element_to_be_clickable((By.ID, "closePCI")))
    ensure_clickable(driver, close_button)
    close_button.click()

    # Wait for page to return to initial state
    wait_for_page_load(wait)

    # Mark the service order as processed
    service_order_df.loc[idx, "processed"] = True


Processing Row 17 for Service Order: 14142811
Clicked 'See service order' button successfully.
Document 2: Uploaded successfully: C:\Users\andmn\OneDrive - Komatsu Ltd\REPARACION\CILINDRO_DIRECCION\YK00487\INFORMES\14142166_292920_INI_APRO_Informe Tecnico Preliminar [Revisado].pdf
Document 5: Uploaded successfully: C:\Users\andmn\OneDrive - Komatsu Ltd\REPARACION\CILINDRO_DIRECCION\YK00487\INFORMES\14142166_292920_FIN_APRO_Informe Tecnico Final [Revisado].pdf

Processing Row 18 for Service Order: 14142812
Clicked 'See service order' button successfully.
Document 2: Uploaded successfully: C:\Users\andmn\OneDrive - Komatsu Ltd\REPARACION\CILINDRO_DIRECCION\WH340\INFORMES\14142811_294413_INI_APRO_Informe Tecnico Preliminar [Revisado].pdf
Document 4: Uploaded successfully: C:\Users\andmn\OneDrive - Komatsu Ltd\REPARACION\CILINDRO_DIRECCION\WH340\INFORMES\14142811_294413_FIN_APRO_Informe Tecnico Final [Revisado].pdf

Processing Row 19 for Service Order: 14141508
Clicked 'See service order'

In [11]:
service_order_df.subcomponent_name.unique()

array(['alternador_principal', 'cilindro_direccion', 'cilindro_levante',
       'motor_traccion', 'suspension_trasera'], dtype=object)

In [5]:
service_order_df

,service_order,equipment_name,customer_work_order,component_serial,changeout_date,subcomponent_name,processed
0,14140838,TK399,18123419,EE13070408,2023-10-30,alternador_principal,True
1,14141425,TK859,18123759,EE13090440,2023-11-08,alternador_principal,False
2,14140979,TK883,18123800,EE14010717,2023-12-01,alternador_principal,False
3,14142021,TK852,18123924,EE19090298,2023-12-09,alternador_principal,False
4,14143179,TK868,18125034,EE14090173,2024-02-07,alternador_principal,False
5,14143655,TK851,18125407,EE14070623,2024-03-14,alternador_principal,False
6,14144274,TK849,18125653,EE14030724,2024-03-28,alternador_principal,False
7,14146810,TK860,18127630,EE11060983A,2024-04-03,alternador_principal,False
8,14144747,TK285,18126171,EE04090417,2024-04-10,alternador_principal,False
9,14144858,TK848,18126402,EE08080858,2024-04-24,alternador_principal,False


In [17]:
processed_df.target_folder

1    01.%20%C3%81REAS%20KCH/1.6%20CONFIABILIDAD/CAE...
Name: target_folder, dtype: object